In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import json
import os
from os.path import join
import allensdk.core.swc as swc
import glob

In [2]:
def to_dict(swc_file, scaling):
    nodes = {}
    with open(swc_file, "r") as f:
        for line in f:
            if line.lstrip().startswith('#'):
                continue
            toks = line.split()
            node_dict = {
                'id' : int(toks[0]),
                'type' : int(toks[1]),
                'x' : float(toks[2])/scaling * 1000,
                'y' : float(toks[3])/scaling * 1000,
                'z' : float(toks[4])/scaling * 1000,
                'radius' : float(toks[5]),
                'parent' : int(toks[6].rstrip())
            }
            nodes[int(toks[0])] = node_dict

    return nodes

def shift(x, y, z, morpho):
	for node in morpho.keys():
		morpho[node]['x'] += x
		morpho[node]['y'] += y
		morpho[node]['z'] += z
		
	return morpho

def dict_to_swc(neuron_dict, filename):
    """
    Takes a neuron dictionary and converts to a swc file
    """
    f = open(filename, 'w')
    f.write("# id,type,x,y,z,r,pid\n")
    for vals in neuron_dict.values():
        f.write("%d %d " % (vals['id'], vals['type']))
        f.write("%.4f " % vals['x'])
        f.write("%.4f " % vals['y'])
        f.write("%.4f " % vals['z'])
        f.write("%.4f " % vals['radius'])
        f.write("%d\n" % vals['parent'])
    f.close()


In [3]:
swc_dir = r'/Users/alice/neuron_morphology_scripts/cell_types_card/swc_files'
outdir = r'/Users/alice/neuron_morphology_scripts/cell_types_card/la_swc'


os.chdir(swc_dir)
files = glob.glob('*.swc')
len(files)

10

In [4]:
# META DATA

meta = pd.read_csv(r'/Users/alice/neuron_morphology_scripts/tolias_m1/m1_patchseq_meta_data.csv', sep='\t')

cells = meta['Cell'].values

depth = meta['Soma depth (µm)'].values
depth[depth=='Slice Lost'] = np.nan
depth = depth.astype(float)
thickness = meta['Cortical thickness (µm)'].values
thickness[thickness==0] = np.nan
thickness = thickness.astype(float)
traced = meta['Traced'].values=='y'
exclude = meta['Exclusion reasons'].values.astype(str)
exclude[exclude=='nan'] = ''

mice_names = meta['Mouse'].values
mice_ages = meta['Mouse age'].values
mice_cres = np.array([c if c[-1]!='+' and c[-1]!='-' else c[:-1] for c in meta['Cre'].values])
mice_ages = dict(zip(mice_names, mice_ages))
mice_cres = dict(zip(mice_names, mice_cres))

print('Number of cells with measured depth:    ', np.sum(~np.isnan(depth)))
print('Number of cells with measured thickness:', np.sum(~np.isnan(thickness)))
print('Number of reconstructed cells:          ', np.sum(traced))

sliceids = meta['Slice'].values
a,b = np.unique(sliceids, return_counts=True)
assert np.all(b<=2)
print('Number of slices with two cells:        ', np.sum(b==2))

# Some consistency checks
assert np.all([np.unique(meta['Date'].values[mice_names==m]).size==1 for m in mice_names])
assert np.all([np.unique(meta['Mouse age'].values[mice_names==m]).size==1 for m in mice_names])
assert np.all([np.unique(meta['Mouse gender'].values[mice_names==m]).size==1 for m in mice_names])
assert np.all([np.unique(meta['Mouse genotype'].values[mice_names==m]).size==1 for m in mice_names])
assert np.all([np.unique(meta['Mouse'].values[sliceids==s]).size==1 for s in sliceids])

Number of cells with measured depth:     1284
Number of cells with measured thickness: 1284
Number of reconstructed cells:           646
Number of slices with two cells:         69


In [5]:
def sroot(m):
    """ Returns root node of soma, if present"""
    if len(m._tree_list) > 0 and m._tree_list[0][0]['type'] == 1:
        return m._tree_list[0][0]
    return None

In [6]:

for f in files:
    #sp = str(f[:-41])
    #print(f)
    
    
    if '.SWC' in f:
        sp = f[:-38]
    else:
        sp = str(f[:-41])
    print("Processing", sp)
    
    #find shrinkage factor
    ix = np.where(cells == sp)
    tthick = thickness[ix]
    tdep = depth[ix]
    #scaling_factor = 
    sdep = tdep/tthick * 1000
    
    mdict = to_dict(join(swc_dir, f), tthick)
    soma_shift = sdep - mdict[1]['y']
    smdict = shift(0, soma_shift, 0, mdict)
    dict_to_swc(smdict, join(outdir, sp + '_fixed.swc'))



Processing 20190619_sample_5
Processing 20171219_sample_6
Processing 20190214_sample_1
Processing 20191030_sample_3
Processing 20180225_sample_5
Processing 20180102_sample_1
Processing 20180118_sample_10
Processing 20190926_sample_2
Processing 20171207_sample_2
Processing 20180123_sample_7


In [7]:
outdir

'/Users/alice/neuron_morphology_scripts/cell_types_card/la_swc'

In [8]:
ext = '.SWC_pruned.swc_connected_smoothed.swc'
len(ext)

38

In [9]:
ext = '_pruned.swc_connected_smoothed_marked.swc'
len(ext)

41

In [10]:
t = '20190619_sample_5.SWC_pruned.swc_connected_smoothed.swc'
t[:-38]

'20190619_sample_5'